In [7]:
# import dependencies
import random
import json


In [8]:
# select desired state
state = 'missouri'

# access json file with data to be sonified for avaliable states
stateData = json.load(open('../resources/stateData.json'))

# list to hold probabilities for which demographic a new case will belong to
infectProbs = []
# list to hold probabilities that an infection will result in death for each demographic
deathProbs = []

# list to hold probabilies that a demographic will gain control of the shared video stream
streamProbs = []

# populate lists above
for data in stateData[state]:
    infectProbs.append(stateData[state][data]['chance_of_infection'])
    deathProbs.append(stateData[state][data]['chance_of_death'])
    streamProbs.append(stateData[state][data]['inverted_ccvi'])


In [10]:
# weighted probabilities for #people the current case can spread infection to
# (from left to right, values refer to weights for 0, 1, 2, and 3 new infections)
spreadProbs = [0,0,0,12]
# symptomatic cases that have occured
sympCount = 0
# deaths that have occured
deathCount = 0
# case count threshold for when new r0 shoud be calculated
thresh = 0
# nth threshold
n = 1
# infection rate
r0 = 3
# transposition in semitones
trans = 0
# cases remaining (one case to start it off)
times = []
# which case
case = 0
# List of midi notes
midiNotes = [48, 51, 55, 58, 60, 62, 63, 65, 67, 68, 70, 74, 77, 80]
# dictionary with all data
weathering = {
    'time': [],
    'demo': [],
    'note': [],
    'death': []
}


In [9]:
# function to determine how many cases the nth threshold is defined by
def nthThresh(n,thresh):
    return 100 * n + thresh

# function to calculate characteristics of new threshold when previous one is past
def newThresh():
    global n
    global thresh
    if n == 1:
        spreadProbs[3] -= 1
        spreadProbs[2] += 1
    elif n == 2:
        spreadProbs[3] -= 1
        spreadProbs[1] += 1
    elif n == 13:
        spreadProbs[2] -= 1
        spreadProbs[0] += 1
    elif n == 14:
        spreadProbs[1] -= 1
        spreadProbs[0] += 1
    elif n != 0 and n < 13:
        spreadProbs[3] -= 1
        spreadProbs[0] += 1
    thresh = nthThresh(n, thresh)
    n += 1

# function to select rescaled incubation period as a delay in milliseconds.
def incubate(n):
    if n == 1:  
        return random.randint(0, 120)*2000/3
    elif n == 2:
        sub = random.choice([3,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 3:
        sub = random.choice([6,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 4:
        sub = random.choice([6,8])
        return random.randint(0, sub*40)*2000/sub
    elif n == 5:
        sub = random.choice([6,8,5])
        return random.randint(0, sub*40)*2000/sub
    else:
        sub = random.choice([6,8,5,7])
        return random.randint(0, sub*40)*2000/sub

# function to select a value based on a probability distribution list
def select(probsList):
    return random.choices(list(range(len(probsList))),weights=probsList)[0]

# function to decide between one and zero based on probability that 1 will be chosen
def decide(prob):
    return random.choices([0,1], weights=[100-prob, prob])[0]

# decide new transposition value (called every 50 cases)
def newTrans():
    global trans
    t = random.randint(1,12)
    if trans+t > 12:
        trans -= t
    elif trans-t < -12:
        trans += t
    else:
        trans += t*random.choice([-1,1])

# function to generate midi notes
def midi(trans, death):
    note = random.choice(midiNotes)
    return note + trans - death



In [11]:
# incubation period of first infection
times.append(incubate(n))

# execute infection model
while len(times) > 0:
    
    # determine how much time has elapsed since first infection
    now = min(times)
    
    # if symptomatic add case info to weathering
    if decide(40) == 0:

        # add current case to total # of cases (only sympotmatic cases count towards threshold)
        sympCount += 1
        
        # 50 new symptomatic cases result in transposition of avaliable notes
        if sympCount % 50 == 0:
            newTrans()

         # if symptomatic case count is at or over current threshold number
        if sympCount > nthThresh(n, thresh):
            # adjust parameters for new threshold
            newThresh()
        
        # add current time to list of when symptomatic cases manifest
        weathering['time'].append(now)

        # select/record which demographic the current case belongs to
        demo = select(infectProbs)
        weathering['demo'].append(demo)

        # decide/record if case results in death or not
        death = decide(deathProbs[demo])
        weathering['death'].append(death)

        # select which note the current case is represented by
        weathering['note'].append(midi(trans, death))

    # decide how many people the current case infects and add them to more
    for i in range(select(spreadProbs)):
        
        # select incubation period (delay time)
        delay = incubate(n)
        
        # determine time elapsed since first infection and append to times list
        time = now + delay
        times.append(time)

    # remove timestamp of current case from remaining cases
    times.pop(times.index(now))


In [12]:
# zip all data into a list a tuples
result = sorted(list(zip(weathering['time'], weathering['demo'], weathering['note'], weathering['death'])))



In [10]:
# for i, r in enumerate(result):
#     print(f'{i}: {r[4]}')



In [125]:
timing = {}
ms = 0
infection = -1

for case in result:
    
    time = round(case[0], 6)

    if time != ms:
        
        if infection != -1:
            
            demoNotes = timing[infection]['demoNotes']
            demoNotes[3:len(demoNotes):4] = [select(demoWeights)]*len(demoNotes[3:len(demoNotes):4])
        

        
        demoWeights = [0,0,0,0,0,0]
        demoWeights[case[1]] += streamProbs[case[1]]
        
        
        infection += 1
        timing[infection] = {'demoNotes': [case[1], case[2], case[3], 'stream']}
        timing[infection]['delay'] = round(time - ms, 6)
        ms = time
        

    else:
        demoWeights[case[1]] += streamProbs[case[1]]
        timing[infection]['demoNotes'].extend(list(case[1:4])+["stream"])


In [126]:
timing



{0: {'demoNotes': [1, 48, 0, 1], 'delay': 37333.333333},
 1: {'demoNotes': [5, 65, 0, 5], 'delay': 52000.0},
 2: {'demoNotes': [5, 63, 0, 5], 'delay': 15333.333334},
 3: {'demoNotes': [0, 60, 0, 0], 'delay': 3333.333333},
 4: {'demoNotes': [2, 63, 0, 2], 'delay': 10000.0},
 5: {'demoNotes': [5, 80, 0, 5], 'delay': 9333.333333},
 6: {'demoNotes': [5, 74, 0, 5], 'delay': 666.666667},
 7: {'demoNotes': [2, 74, 0, 2], 'delay': 2000.0},
 8: {'demoNotes': [0, 70, 0, 0], 'delay': 666.666667},
 9: {'demoNotes': [5, 74, 0, 5], 'delay': 666.666666},
 10: {'demoNotes': [0, 48, 0, 0], 'delay': 666.666667},
 11: {'demoNotes': [1, 60, 0, 1], 'delay': 6666.666667},
 12: {'demoNotes': [1, 58, 0, 1], 'delay': 1333.333333},
 13: {'demoNotes': [0, 51, 0, 0], 'delay': 6666.666667},
 14: {'demoNotes': [5, 55, 0, 5], 'delay': 1333.333333},
 15: {'demoNotes': [0, 60, 0, 0, 0, 51, 0, 0], 'delay': 1333.333333},
 16: {'demoNotes': [5, 51, 0, 5], 'delay': 1333.333334},
 17: {'demoNotes': [2, 62, 0, 2, 5, 70, 0, 

In [ ]:
with open("../resources/weathering.json", "w") as outfile:
    json.dump(timing, outfile)
    